In [1]:
from comet_ml import Experiment
from scikeras.wrappers import KerasClassifier
from keras_metrics_module import f1_1_func, f1_0_func, recall_func, precision_func, f1_macro_func
import itertools
import pandas as pd
from modules.testing_module import metrics_report
from modules import json_module, h5py_module, dirs_module
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, ReLU
import numpy as np
import tensorflow as tf
import keras_tuner as kt


dirs_module.create_directory('saved_models', warn_exists = False)


In [2]:
HZ = '32kHz'
SPLIT_ID = 'S3'
REGIME = ''
FEATURE = f'HostService_{HZ}_vibration_1'
DURATION = '01S'
ADDITIONAL_INFO = 'CNN'
COMET_PROJECT = 'bayesian'
EXPERIMENT_ID = f'{SPLIT_ID}_{DURATION}_{FEATURE}_{REGIME}_{ADDITIONAL_INFO}'

data_dir = r'data\s3_01s_oversampling001005_32HZ'

healthy_train = ['K001','K002','K003']
real_damage_train = ['KA04','KA15','KA22','KA30','KB23','KB27','KI04','KI17']
artificial_damage_train = ['KA01','KA05','KA07','KI01','KI03']
train_bearing_codes = healthy_train + artificial_damage_train + real_damage_train

In [3]:
reading_files_hz = '64kHz'
first_read = True
for bearing_code in train_bearing_codes:

    if first_read:
        first_read = False
        x_train = np.load(f'{data_dir}/x_train_{REGIME}{bearing_code}_{FEATURE.replace(HZ,reading_files_hz)}.npy', allow_pickle = True)
        y_train = np.load(f'{data_dir}/y_train_{REGIME}{bearing_code}.npy', allow_pickle = True)
    else:
        x_train = np.concatenate(
            [x_train, np.load(f'{data_dir}/x_train_{REGIME}{bearing_code}_{FEATURE.replace(HZ,reading_files_hz)}.npy', allow_pickle = True)],
            axis = 0)
        y_train = np.concatenate(
            [y_train, np.load(f'{data_dir}/y_train_{REGIME}{bearing_code}.npy', allow_pickle = True)],
            axis = 0)

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
INPUT_SHAPE = (x_train.shape[1], x_train.shape[2])
x_train.shape

(426480, 3200, 1)

In [4]:
x_test = np.load(f'{data_dir}/x_test_{REGIME}{FEATURE.replace(HZ,reading_files_hz)}.npy', allow_pickle = True)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
y_test = np.load(f'{data_dir}/y_test{REGIME}.npy', allow_pickle = True)
x_test.shape

(40521, 3200, 1)

In [5]:
x_val = np.load(f'{data_dir}/x_val_{REGIME}{FEATURE.replace(HZ,reading_files_hz)}.npy', allow_pickle = True)
x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))
y_val = np.load(f'{data_dir}/y_val{REGIME}.npy', allow_pickle = True)
x_val.shape

(121840, 3200, 1)

In [6]:
y_value_counts = pd.Series(list(y_train)).value_counts()
class_weight = {
    0: y_value_counts[1] / y_value_counts[0],
    1: 1
}
y_value_counts

0    228360
1    198120
dtype: int64

In [7]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
y_val = tf.keras.utils.to_categorical(y_val)

In [8]:
search_space = dict(
    wide_layer_kernel_size = [16,32],
    wide_layer_num_kernels = [16,32],
    num_narrow_cnn_layers = [2,3],
    narrow_layers_kernel_size = [8,16],
    narrow_layers_num_kernels = [16,32,64],

    num_dense_layers = [2],
    dense_layers_num_nodes = [64],
    dense_layers_shrinkage_factor = [0.5],

    dropout = [True],
    batch_norm = [True],

    batch_size = [256],
    loss_name = ['binary_crossentropy', 'categorical_hinge'],
    optimizer_name = ['adam'],
    learning_rate = [0.01, 0.25, 0.5],
    epochs = [5],
)

In [11]:
def build_model(hp):

    wide_layer_kernel_size = hp.Int('wide_layer_num_kernels',2,512)
    wide_layer_num_kernels = hp.Int('wide_layer_num_kernels',4,64)
    # num_narrow_cnn_layers = hp.Int('num_narrow_cnn_layers',1,5)
    # narrow_layers_kernel_size = hp.Int('narrow_layers_kernel_size',4,128)
    # narrow_layers_num_kernels = hp.Int('narrow_layers_num_kernels',4,64)
    num_dense_layers = hp.Int('num_dense_layers',1,3)
    dense_layers_num_nodes = hp.Int('dense_layers_num_nodes',16,128)
    dense_layers_shrinkage_factor = 0.75
    optimizer_name = 'adam'
    learning_rate = hp.Choice('learning_rate', [0.001, 0.01, 0.1, 0.25, 0.5])
    loss_name = 'categorical_crossentropy'
    dropout = False
    batch_norm = hp.Choice('batch_norm', [True, False])

    layers_list = list()
    layers_list.append(
        Conv1D(filters=wide_layer_num_kernels,
           kernel_size=wide_layer_kernel_size,
           # strides = int(wide_layer_kernel_size/4),
           input_shape=INPUT_SHAPE,
           padding = 'same')
    )
    layers_list.append(ReLU())
    if batch_norm:
        layers_list.append(BatchNormalization())
    layers_list.append(MaxPooling1D(pool_size=4, padding="same"))

    # for narrow_layer_number in range(1,num_narrow_cnn_layers+1):
    #     layers_list.append(
    #         Conv1D(filters=narrow_layers_num_kernels,
    #                kernel_size=(narrow_layers_kernel_size,),
    #                strides = int(wide_layer_kernel_size/2),
    #                padding = 'same')
    #     )
    #     layers_list.append(ReLU())
    #     if batch_norm:
    #         layers_list.append(BatchNormalization())
    #     layers_list.append(MaxPooling1D(pool_size=2, padding="same"))

    layers_list.append(Flatten())

    dense_layer_current_num_nodes = dense_layers_num_nodes * (1/dense_layers_shrinkage_factor)
    for dense_layer_number in range(1,num_dense_layers+1):
        dense_layer_current_num_nodes = dense_layer_current_num_nodes * dense_layers_shrinkage_factor
        layers_list.append(Dense(dense_layer_current_num_nodes))
        layers_list.append(ReLU())
        if batch_norm:
            layers_list.append(BatchNormalization())
        if dropout:
            layers_list.append(Dropout(0.1))
    if dropout:
        layers_list = layers_list[:-1]


    layers_list.append(Dense(2, activation='softmax'))

    if optimizer_name.lower() == 'adam':
        optimizer = tf.optimizers.Adam(learning_rate = learning_rate)

    built_model = Sequential(layers_list)
    built_model.compile(loss=loss_name, optimizer=optimizer,
                        metrics=['accuracy', f1_macro_func, f1_1_func, f1_0_func, recall_func, precision_func])
    tf.keras.utils.plot_model(built_model, to_file='model.pdf', show_shapes=True)
    return built_model

In [1]:
# try:
experiment = Experiment(
    api_key = 'A8Lg71j9LtIrsv0deBA0DVGcR',
    project_name = COMET_PROJECT,
    workspace = 'diploma',
    auto_output_logging = 'native',
)
experiment.set_name(f"Bayesian")
experiment.add_tags(EXPERIMENT_ID.split('_'))

tuner = kt.BayesianOptimization(
    build_model,
    objective = kt.Objective(f1_macro_func, direction="max"),
    max_trials=1000,
    executions_per_trial=2,
    directory="keras_tuner_logs",
    project_name="initial_val",
)

with experiment.train():
    tuner.search(x_train, y_train,
                 epochs = 10,
                 class_weight = class_weight,
                 verbose = 10,
                 validation_data = (x_val, y_val))



In [ ]:
results_list = list()
models_dir = 'saved_models'
import os
for model_file in os.listdir(models_dir):
    model_i = tf.keras.models.load_model(f'{models_dir}/{model_file}',
            custom_objects = {
                'f1_macro_func':f1_macro_func,
                'f1_1_func':f1_1_func,
                'f1_0_func':f1_0_func,
                'precision_func':precision_func,
                'recall_func':recall_func
             })

    y_pred = model_i.predict(x_test)
    y_pred = np.where(y_pred > 0.5, 1, 0).ravel()

    test_metrics = metrics_report(y_test, y_pred)
    print(test_metrics)
    results_list.append(test_metrics)

    for layer_i in model_i.layers:
        try:
            print(layer_i.kernel_size)
        except:
            pass

In [ ]:
pd.DataFrame(results_list).to_excel('results.xlsx')